<a href="https://colab.research.google.com/github/Nanashi-bot/eduquery/blob/main/Eduquery_prototype1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

## Chat with eBook
### Installation

In [ ]:
!pip install langchain sentence-transformers chromadb pypdf unstructured pdf2image

In [ ]:
!pip install unstructured['pdf']

In [1]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

In [2]:
import os
from getpass import getpass

HF_token = getpass()

··········


In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

In [6]:
from langchain_community.document_loaders import UnstructuredFileLoader

In [7]:
data = UnstructuredFileLoader("./ME1.txt")

In [4]:
file_path = "ME1.txt"

In [8]:
content = data.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
print(content)

[Document(page_content="income inequality lorez,gini and quintiles\n\nin this section of the course what we're\n\ntalking about is income inequality in\n\nother words why is it that some people\n\nmake more than others and as a part of\n\nthis topic we're going to be talking\n\nabout poverty what it means how its\n\nmeasured and discrimination as one of\n\nthe reasons why some people make more\n\nthan others the first thing we have to\n\ntalk about before we talk about anything\n\nabout income inequality is that when we\n\ndiscuss things like poverty and income\n\ninequality we're not usually talking\n\nabout individuals and how their incomes\n\nare different we're usually talking\n\nabout households so let's first talk for\n\na minute about what a household is so\n\nwhen we talk about an income\n\ndistribution how some people make more\n\nthan others and we're going to look at\n\none of these in a few different ways in\n\na minute we're talking about household\n\nincome and equality s

In [10]:
len(content[0].page_content)

36454

## Chunking- Text Splitter

In [11]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=0)

## Embedding Model

In [12]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "thenlper/gte-large"
)

In [13]:
vectorstore = Chroma(embedding_function=embeddings)
store = InMemoryStore()

In [15]:
model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       model_kwargs={"temperature":0.8,"max_new_tokens":512,"max_length":64})

In [16]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [17]:
retriever.add_documents(content,ids=None)

In [39]:
query = "give a summary for the income inequality "

In [40]:
relevant_context = retriever.get_relevant_documents(query)

In [41]:
relevant_context

[Document(page_content="as a lot of very high income people live\n\nin this county and lastly let's look at\n\nincome inequality over time in the\n\nUnited States it's kind of hard to tell\n\nwhat these numbers actually mean the a\n\nlot of people say the income inequality\n\nin the United States is getting worse\n\nand looking at the Gini coefficients\n\nthey are they're getting higher nineteen\n\nseventy point three nine two thousand\n\npoint four six now in 2007 the income\n\ninequality actually went down and then", metadata={'source': './ME1.txt'}),
 Document(page_content="out a couple of things number one\n\nremember that the household income is\n\nwhat we use to calculate income\n\ninequality what has happened to the\n\nnumber of people in the average\n\nhousehold over these forty five years\n\nwell the average number of people in a\n\nhousehold has gone down from around\n\nthree and a half-ish to about two and a\n\nhalf and if you have less people living\n\nin a household you're

In [42]:
print(relevant_context)

[Document(page_content="as a lot of very high income people live\n\nin this county and lastly let's look at\n\nincome inequality over time in the\n\nUnited States it's kind of hard to tell\n\nwhat these numbers actually mean the a\n\nlot of people say the income inequality\n\nin the United States is getting worse\n\nand looking at the Gini coefficients\n\nthey are they're getting higher nineteen\n\nseventy point three nine two thousand\n\npoint four six now in 2007 the income\n\ninequality actually went down and then", metadata={'source': './ME1.txt'}), Document(page_content="out a couple of things number one\n\nremember that the household income is\n\nwhat we use to calculate income\n\ninequality what has happened to the\n\nnumber of people in the average\n\nhousehold over these forty five years\n\nwell the average number of people in a\n\nhousehold has gone down from around\n\nthree and a half-ish to about two and a\n\nhalf and if you have less people living\n\nin a household you're 

In [21]:
query = "explain me Gini coefficeints"

In [22]:
relevant_context = retriever.get_relevant_documents(query)

In [23]:
relevant_context

[Document(page_content="would like to find some kind of function\n\nhere that describes that Gini\n\ncoefficient curves so here we have a\n\npolynomial with an X to the fourth X to\n\nthe third x squared and X term in it and\n\nthen we could use some fancy math in\n\norder to calculate those areas but I\n\nalso want to show you what happens to\n\nthis Lorenz curve this Gini coefficient\n\nwhen we take into account some things\n\nthat aren't normally taking into account\n\nso when we talk about income and we ask\n\npeople these questions how much money", metadata={'source': './ME1.txt'}),
 Document(page_content="lists come from the 1990s and some of it\n\nfrom the 2000s or 2010's in some of\n\nthese countries it's kind of hard to get\n\nincome data that's very believable or\n\nvery recent so they just use the data\n\nthat they can find pretty easily so\n\nlet's have a look at a few of these Gini\n\ncoefficients from the CIA World Factbook\n\nas we were talking about Lesotho South\n\nAfr

In [24]:
!pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.8 MB/s eta 0:00:00


## Step-1 Retrieval

In [43]:
os.environ["COHERE_API_KEY"] = "5uuX8mk9dhf9KHzw7vSDhQdXlV2x92MzELvJ972T"

In [44]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [45]:
from cohere import Client

In [58]:
co = Client(api_key = "5uuX8mk9dhf9KHzw7vSDhQdXlV2x92MzELvJ972T")

In [46]:
from typing import ForwardRef
from pydantic import BaseModel

class CustomCohereRerank(CohereRerank):
  class Config(BaseModel.Config):
    arbitrary_types_allowed = True

CustomCohereRerank.update_forward_refs()

AttributeError: Config

In [37]:
CustomCohereRerank.update_forward_refs()

NameError: name 'CustomCohereRerank' is not defined

In [36]:
compressor = CustomCohereRerank(client=co)

NameError: name 'CustomCohereRerank' is not defined

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

## Step - 2 Augment

In [61]:
from langchain_core.prompts import ChatPromptTemplate

In [62]:
template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [63]:
prompt = ChatPromptTemplate.from_template(template)

## Step-3 Generation

In [64]:
from langchain_core.output_parsers import StrOutputParser

In [65]:
from langchain_core.runnables import RunnablePassthrough

In [66]:
output_parser = StrOutputParser()

In [67]:
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [68]:
query = "Who is Rahul?"

In [69]:
response = chain.invoke(query)

In [70]:
print(response)

Human: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: [Document(page_content="bottom four groups the bottom 80% are\n\nall very poor they're very equal but\n\nthen most of the income eighty-four\n\npercent of the total is had by just this\n\ntop twenty percent of elites all right\n\nwell let's calculate this cumulative\n\ndistribution and graph it and let's see\n\nwhat it looks like so we always start a\n\ncumulative distribution with just the\n\npercent in the bottom category here so\n\nit's going to be 4 percent saying that\n\nin this first category the poorest 20%", metadata={'source': './ME1.txt'}), Document(page_content="we're going to take the bottom 20% of\n\nhouseholds the lowest income 20% and\n\nthen the next 20% so if you look at this\n\ntable here the we might call them the\n\npoorest 20% lower middle 20% the middle\n\n20% the uppe

In [74]:
print(chain.invoke("what is a housing unit"))

Human: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: [Document(page_content="get going but a brief definition is that\n\na household includes all the persons who\n\noccupy a housing unit\n\nwhat's a housing unit well a housing\n\nunit is a house an apartment a mobile\n\nhome a group of rooms or a single room\n\nthat is occupied or if vacant is\n\nintended for occupancy as separate\n\nliving quarters so it could be a house\n\nan apartment a mobile home or it could\n\neven be a dorm room so when we start\n\nthinking about some households making\n\nmore than others we have to include all", metadata={'source': './ME1.txt'}), Document(page_content="discuss things like poverty and income\n\ninequality we're not usually talking\n\nabout individuals and how their incomes\n\nare different we're usually talking\n\nabout households so let's first talk f